In [1]:
import cv2
import numpy as np
import random
import os


input_filename = 'img/a.jpg'
output_dir = 'output_cau1'

# Tạo thư mục output nếu nó chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Đã tạo thư mục '{output_dir}'")

try:
    image = cv2.imread(input_filename)
    if image is None:
        raise FileNotFoundError(f"Không thể đọc được file {input_filename}. Hãy chắc chắn rằng file tồn tại và ở đúng đường dẫn.")
except FileNotFoundError as e:
    print(e)
    exit()

# --- Yêu cầu 1: Sử dụng bộ lọc trung bình (Mean Filter) ---
print("Đang xử lý: Bộ lọc trung bình...")
mean_filtered_image = cv2.blur(image, (5, 5))
output_path = os.path.join(output_dir, 'a_mean_filter.jpg')
cv2.imwrite(output_path, mean_filtered_image)
print(f"Đã lưu ảnh lọc trung bình vào: {output_path}")

# --- Yêu cầu 2: Sử dụng bộ lọc để xác định biên (Edge Detection) ---
print("Đang xử lý: Phát hiện biên...")
# Sử dụng thuật toán Canny để phát hiện biên. [1, 4]
# 100 và 200 là ngưỡng dưới và trên. [4]
edges = cv2.Canny(image, 100, 200)
output_path = os.path.join(output_dir, 'a_edges.jpg')
cv2.imwrite(output_path, edges)
print(f"Đã lưu ảnh xác định biên vào: {output_path}")

# --- Yêu cầu 3: Đổi màu ảnh BGR sang màu RGB ngẫu nhiên ---
print("Đang xử lý: Đổi màu ngẫu nhiên...")
# OpenCV đọc ảnh mặc định theo định dạng BGR. [18]
random_color_image = image.copy()
b, g, r = cv2.split(random_color_image)
channels = [b, g, r]
random.shuffle(channels)
random_color_image = cv2.merge(channels)
output_path = os.path.join(output_dir, 'a_random_color.jpg')
cv2.imwrite(output_path, random_color_image)
print(f"Đã lưu ảnh đổi màu ngẫu nhiên vào: {output_path}")

# --- Yêu cầu 4: Chuyển sang không gian màu HSV và tách kênh ---
print("Đang xử lý: Tách kênh HSV...")
# Chuyển đổi ảnh từ không gian màu BGR sang HSV (Hue, Saturation, Value). [8, 13]
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# Tách các kênh Hue, Saturation, và Value. [7, 9]
h, s, v = cv2.split(hsv_image)

# Lưu từng kênh dưới dạng ảnh grayscale
path_h = os.path.join(output_dir, 'a_hue.jpg')
path_s = os.path.join(output_dir, 'a_saturation.jpg')
path_v = os.path.join(output_dir, 'a_value.jpg')

cv2.imwrite(path_h, h)
cv2.imwrite(path_s, s)
cv2.imwrite(path_v, v)
print(f"Đã lưu kênh Hue vào: {path_h}")
print(f"Đã lưu kênh Value vào: {path_v}")

print(f"\nHoàn thành tất cả các yêu cầu! Tất cả các file đã được lưu trong thư mục '{output_dir}'.")

Đang xử lý: Bộ lọc trung bình...
Đã lưu ảnh lọc trung bình vào: output_cau1\a_mean_filter.jpg
Đang xử lý: Phát hiện biên...
Đã lưu ảnh xác định biên vào: output_cau1\a_edges.jpg
Đang xử lý: Đổi màu ngẫu nhiên...
Đã lưu ảnh đổi màu ngẫu nhiên vào: output_cau1\a_random_color.jpg
Đang xử lý: Tách kênh HSV...
Đã lưu kênh Hue vào: output_cau1\a_hue.jpg
Đã lưu kênh Value vào: output_cau1\a_value.jpg

Hoàn thành tất cả các yêu cầu! Tất cả các file đã được lưu trong thư mục 'output_cau1'.


In [5]:
import cv2
import numpy as np
import os
import random
import ipywidgets as widgets
from IPython.display import display, clear_output


# THAY ĐỔI 1: Khai báo thư mục chứa ảnh
input_dir = 'img' 

# Danh sách các tệp ảnh đầu vào (chỉ cần tên file)
input_filenames = ['image1.jpg', 'image2.jpg', 'image3.jpg']

# Thư mục để lưu tất cả các kết quả
output_dir = 'output_cau2'

# Tự động tạo thư mục output nếu nó chưa tồn tại
os.makedirs(output_dir, exist_ok=True)

# THAY ĐỔI 2: Cập nhật phần kiểm tra file để tìm trong đúng thư mục
print("Kiểm tra các file ảnh đầu vào...")
for filename in input_filenames:
    full_path = os.path.join(input_dir, filename) # Tạo đường dẫn đầy đủ
    if not os.path.exists(full_path):
        print(f"!!! CẢNH BÁO: Không tìm thấy file tại '{full_path}'.")
print("Hoàn tất kiểm tra.")


def inverse_transform(img):
    return cv2.bitwise_not(img)

def gamma_correction(img, gamma):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)

def log_transform(img, c):
    log_image = c * np.log1p(img.astype(np.float32))
    cv2.normalize(log_image, log_image, 0, 255, cv2.NORM_MINMAX)
    return log_image.astype("uint8")

def histogram_equalization(img):
    return cv2.equalizeHist(img)

def contrast_stretching(img, min_val, max_val):
    stretched = img.copy().astype(np.float32)
    current_min, current_max = np.min(stretched), np.max(stretched)
    if current_max - current_min == 0: return img
    stretched = (stretched - current_min) * (max_val - min_val) / (current_max - current_min) + min_val
    stretched = np.clip(stretched, 0, 255)
    return stretched.astype("uint8")

def adaptive_histogram_equalization(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(img)


dropdown = widgets.Dropdown(
    options=[
        ('Chọn một phương pháp...', 'none'),
        ('I: Image Inverse Transformation', 'inverse'),
        ('G: Gamma Correction', 'gamma'),
        ('L: Log Transformation', 'log'),
        ('H: Histogram Equalization', 'hist_eq'),
        ('C: Contrast Stretching', 'contrast_stretch'),
        ('A: Adaptive Histogram Equalization (CLAHE)', 'clahe')
    ],
    value='none',
    description='Phương pháp:',
    style={'description_width': 'initial'}
)

button = widgets.Button(
    description="Thực thi cho tất cả ảnh",
    button_style='primary',
    tooltip='Áp dụng phép biến đổi đã chọn lên tất cả ảnh trong danh sách',
    icon='play'
)

out = widgets.Output()


def process_all_images(b):
    choice = dropdown.value
    
    with out:
        clear_output(wait=True)
        
        if choice == 'none':
            print("Vui lòng chọn một phương pháp từ menu rồi nhấn nút.")
            return

        print(f"Bắt đầu xử lý với phương pháp: '{choice.upper()}'")
        
        for i, filename in enumerate(input_filenames):
            # THAY ĐỔI 3: Tạo đường dẫn đầy đủ để đọc ảnh
            full_path = os.path.join(input_dir, filename)
            image = cv2.imread(full_path)
            
            if image is None:
                print(f"-> Bỏ qua: Không thể đọc file '{full_path}'")
                continue

            # Phần logic xử lý ảnh còn lại không thay đổi
            output_image = None
            is_color = len(image.shape) == 3
            
            if choice == 'inverse':
                output_image = inverse_transform(image)
            else:
                channel_to_process = None
                if is_color:
                    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                    h, s, v = cv2.split(hsv)
                    channel_to_process = v
                else:
                    channel_to_process = image
                
                processed_channel = None
                if choice == 'gamma':
                    gamma = random.uniform(0.5, 2.0)
                    processed_channel = gamma_correction(channel_to_process, gamma)
                elif choice == 'log':
                    c = random.uniform(1.0, 5.0)
                    processed_channel = log_transform(channel_to_process, c)
                elif choice == 'hist_eq':
                    processed_channel = histogram_equalization(channel_to_process)
                elif choice == 'contrast_stretch':
                    r1, r2 = random.randint(0, 100), random.randint(155, 255)
                    processed_channel = contrast_stretching(channel_to_process, r1, r2)
                elif choice == 'clahe':
                    processed_channel = adaptive_histogram_equalization(channel_to_process)
                
                if is_color and processed_channel is not None:
                    final_hsv = cv2.merge([h, s, processed_channel])
                    output_image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
                else:
                    output_image = processed_channel

            if output_image is not None:
                # Lấy tên file gốc không có phần mở rộng để tạo tên file output
                base_name = os.path.splitext(filename)[0]
                output_filename = f"output_{choice}_{base_name}.jpg"
                output_path = os.path.join(output_dir, output_filename)
                cv2.imwrite(output_path, output_image)
                print(f"-> Đã xử lý '{full_path}' và lưu tại: {output_path}")

        print("\nHoàn tất xử lý cho tất cả các ảnh!")

button.on_click(process_all_images)

print("CHƯƠNG TRÌNH BIẾN ĐỔI NHIỀU ẢNH CÙNG LÚC")
display(dropdown, button, out)

Kiểm tra các file ảnh đầu vào...
Hoàn tất kiểm tra.
CHƯƠNG TRÌNH BIẾN ĐỔI NHIỀU ẢNH CÙNG LÚC


Dropdown(description='Phương pháp:', options=(('Chọn một phương pháp...', 'none'), ('I: Image Inverse Transfor…

Button(button_style='primary', description='Thực thi cho tất cả ảnh', icon='play', style=ButtonStyle(), toolti…

Output()

In [7]:
import cv2
import numpy as np
import os



# Tên các file ảnh đầu vào
img_fruits_name = 'img/colorful-ripe-tropical-fruits.jpg'
img_qn_name = 'img/quang_ninh.jpg'
img_pagoda_name = 'img/pagoda.jpg'

# Thư mục để lưu tất cả các kết quả
output_dir = 'output_cau3'
os.makedirs(output_dir, exist_ok=True) 



print("Bắt đầu xử lý các ảnh...")

# --- Yêu cầu 1: Tăng kích thước ảnh fruits thêm 30 pixel (0.5 Điểm) ---
try:
    img_fruits = cv2.imread(img_fruits_name)
    if img_fruits is None: raise FileNotFoundError
    
    # Lấy kích thước hiện tại (cao, rộng)
    h, w = img_fruits.shape[:2]
    
    # Kích thước mới
    new_w, new_h = w + 30, h + 30
    
    # Thay đổi kích thước ảnh
    resized_fruits = cv2.resize(img_fruits, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    
    # Lưu kết quả
    output_path = os.path.join(output_dir, 'fruits_resized.jpg')
    cv2.imwrite(output_path, resized_fruits)
    print(f"1. Đã tăng kích thước ảnh fruits và lưu tại: {output_path}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{img_fruits_name}'. Bỏ qua yêu cầu 1.")


# --- Yêu cầu 2: Xoay và lật ảnh quang-ninh (0.5 Điểm) ---
try:
    img_qn = cv2.imread(img_qn_name)
    if img_qn is None: raise FileNotFoundError

    h, w = img_qn.shape[:2]
    center = (w // 2, h // 2) # Tìm tâm của ảnh

    # Tạo ma trận xoay 45 độ theo chiều kim đồng hồ (góc âm)
    rotation_matrix = cv2.getRotationMatrix2D(center=center, angle=-45, scale=1.0)
    
    # Thực hiện xoay ảnh
    rotated_qn = cv2.warpAffine(img_qn, rotation_matrix, (w, h))
    
    # Lật ngang ảnh đã xoay (flip code = 1)
    flipped_rotated_qn = cv2.flip(rotated_qn, 1)

    # Lưu kết quả
    output_path = os.path.join(output_dir, 'quang-ninh_rotated_flipped.jpg')
    cv2.imwrite(output_path, flipped_rotated_qn)
    print(f"2. Đã xoay và lật ảnh quang-ninh và lưu tại: {output_path}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{img_qn_name}'. Bỏ qua yêu cầu 2.")


# --- Yêu cầu 3 & 4: Xử lý ảnh pagoda ---
try:
    img_pagoda = cv2.imread(img_pagoda_name)
    if img_pagoda is None: raise FileNotFoundError

    # --- Yêu cầu 3: Tăng kích thước 5 lần và làm mịn (0.5 Điểm) ---
    # Tăng kích thước lên 5 lần ở cả hai chiều
    upscaled_pagoda = cv2.resize(img_pagoda, None, fx=5.0, fy=5.0, interpolation=cv2.INTER_CUBIC)
    
    # Áp dụng bộ lọc Gaussian blur với kernel 7x7 để làm mịn
    blurred_pagoda = cv2.GaussianBlur(upscaled_pagoda, (7, 7), 0)
    
    # Lưu kết quả
    output_path = os.path.join(output_dir, 'pagoda_upscaled_blurred.jpg')
    cv2.imwrite(output_path, blurred_pagoda)
    print(f"3. Đã tăng kích thước và làm mịn ảnh pagoda, lưu tại: {output_path}")

    # --- Yêu cầu 4: Áp dụng công thức biến đổi độ sáng/tương phản (1.5 Điểm) ---
    # Sử dụng công thức: I_out = α * I_in(a,b) + β
    # Chọn giá trị alpha và beta (tăng tương phản và độ sáng)
    alpha = 1.5  # Hệ số tương phản (contrast)
    beta = 20    # Độ lệch sáng (brightness)
    
    # cv2.convertScaleAbs là hàm tối ưu để thực hiện phép toán:
    # G = α*F + β
    # Hàm này tự động thực hiện phép nhân, cộng và giới hạn giá trị trong khoảng [0, 255]
    adjusted_pagoda = cv2.convertScaleAbs(img_pagoda, alpha=alpha, beta=beta)

    # Lưu kết quả
    output_path = os.path.join(output_dir, f'pagoda_adjusted_alpha{alpha}_beta{beta}.jpg')
    cv2.imwrite(output_path, adjusted_pagoda)
    print(f"4. Đã điều chỉnh độ sáng/tương phản ảnh pagoda và lưu tại: {output_path}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{img_pagoda_name}'. Bỏ qua yêu cầu 3 và 4.")

print("\nHoàn thành tất cả các yêu cầu!")



Bắt đầu xử lý các ảnh...
1. Đã tăng kích thước ảnh fruits và lưu tại: output_cau3\fruits_resized.jpg
2. Đã xoay và lật ảnh quang-ninh và lưu tại: output_cau3\quang-ninh_rotated_flipped.jpg
3. Đã tăng kích thước và làm mịn ảnh pagoda, lưu tại: output_cau3\pagoda_upscaled_blurred.jpg
4. Đã điều chỉnh độ sáng/tương phản ảnh pagoda và lưu tại: output_cau3\pagoda_adjusted_alpha1.5_beta20.jpg

Hoàn thành tất cả các yêu cầu!
